In [1]:
include("../src/ctpPOP.jl")
using .ctpPOP, PowerModels

data = PowerModels.parse_file("/home/hoanganh/Desktop/math-topics/ctpPOP/codes/OPFproblems/pglib_opf_case3_lmbd.m")

using DynamicPolynomials

x,f,g,h=ctpPOP.get_CTP_POP_OPF(data);

println("Number of variable: n=",length(x))
println("Number of inequality constraints: m=",length(g))
println("Number of equality constraints: l=",length(h))
println("====================")

k=2; 
println("Relaxed order: k=",k)
t=2;
println("Sparse order: t=",t)

[info | PowerModels]: removing 3 cost terms from generator 3: Float64[]
[warn | PowerModels]: Updated generator 3 cost function with order 0 to a function of order 3: [0.0, 0.0, 0.0]
Number of variable: n=11
Number of inequality constraints: m=11
Number of equality constraints: l=6
Relaxed order: k=2
Sparse order: t=2


In [5]:
x

11-element Array{PolyVar{true},1}:
 mvr2
 mvi2
 mvr3
 mvi3
 mvr1
 mvi1
 mpg2
 mqg2
 mqg3
 mpg1
 mqg1

In [2]:
f

85000.00000000001mpg2² + 110000.0mpg1² + 171200.00000000003mpg2 + 225000.0mpg1 + 201200.0

In [3]:
g

11-element Array{Polynomial{true,Float64},1}:
 mvr2² + mvi2² - 0.6694214876033058
 -mvr2² - mvi2² + 1.0              
 mvr3² + mvi3² - 0.6694214876033058
 -mvr3² - mvi3² + 1.0              
 mvr1² + mvi1² - 0.6694214876033058
 -mvr1² - mvi1² + 1.0              
 -mpg2² + 1.0                      
 -mqg2² + 1.0                      
 -mqg3² + 1.0                      
 -mpg1² + 1.0                      
 -mqg1² + 1.0                      

In [4]:
h

6-element Array{Polynomial{true,Float64},1}:
 0.11632249327468655mvr2² - 0.05371809100998891mvr2mvr3 - 1.6115427302996674mvr2mvi3 - 0.06260440226469764mvr2mvr1 - 1.341522905672092mvr2mvi1 + 0.11632249327468655mvi2² + 1.6115427302996674mvi2mvr3 - 0.05371809100998891mvi2mvi3 + 1.341522905672092mvi2mvr1 - 0.06260440226469764mvi2mvi1 - 10.0mpg2 - 8.9
 2.3480656359717593mvr2² - 1.6115427302996674mvr2mvr3 + 0.05371809100998891mvr2mvi3 - 1.341522905672092mvr2mvr1 + 0.06260440226469764mvr2mvi1 + 2.3480656359717593mvi2² - 0.05371809100998891mvi2mvr3 - 1.6115427302996674mvi2mvi3 - 0.06260440226469764mvi2mvr1 - 1.341522905672092mvi2mvi1 - 10.0mqg2 + 0.4  
 -0.05371809100998891mvr2mvr3 + 1.6115427302996674mvr2mvi3 - 1.6115427302996674mvi2mvr3 - 0.05371809100998891mvi2mvi3 + 0.25609827756515136mvr3² - 0.20238018655516246mvr3mvr1 - 1.930395625603088mvr3mvi1 + 0.25609827756515136mvi3² + 1.930395625603088mvi3mvr1 - 0.20238018655516246mvi3mvi1 + 0.95         
 -1.6115427302996674mvr2mvr3 - 0.0537180910

In [6]:
using  SparseArrays, LinearAlgebra

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [7]:
using TSSOS, LinearAlgebra


@time begin
    ~,~,data1=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false)
    #=for j in 1:t-1
        ~,~,data1=cs_tssos_higher!(data,TS="block");
    end=#
    end;

------------------------------------------------------
The clique sizes of varibles:
[7]
[5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 414             
  Cones                  : 0               
  Scalar variables       : 190             
  Matrix variables       : 37              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.

In [12]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=false)

------------------------------------------------------
The clique sizes of varibles:
[7]
[5]
------------------------------------------------------
  Number of cliques: p=5
  Largest clique size: u=7


  Largest constant trace: a=21.0
  Number of blocks: omega=56
  Number of equality consraints: zeta=2209
  Size of the largest block: s^max=24
Modeling time:
  0.997632 seconds (1.30 M allocations: 104.819 MiB, 5.89% gc time)
iter=1   val=-4.636689118116671e6   gap=4.017187048910496e-16   feas=56.099839614227676   time=0.005989217
iter=3   val=-980467.2138297629   gap=0.7915729783436992   feas=24.21946313905481   time=0.019574368
iter=7   val=-336563.52431368706   gap=0.9209057885669651   feas=18.989209084699834   time=0.045712967
iter=15   val=-58680.47586300078   gap=0.9743399773037513   feas=10.178883979657623   time=0.101465654
iter=31   val=35050.34432833734   gap=1.0252425334433117   feas=5.325445841777881   time=0.255679065
iter=63   val=3067.1971553819144   gap=1.0038688950572303   feas=3.258230268999411   time=0.39342540699999995
iter=127   val=13461.417077907528   gap=1.0300309862828283   feas=1.537538925137221   time=0.6618688049999999
iter=255   val=25536.63332094099   gap=

InterruptException: InterruptException:

In [10]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,set_initial_point_opf=true)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      113
Number of nonzeros in inequality constraint Jacobian.:       17
Number of nonzeros in Lagrangian Hessian.............:       79

Total number of variables............................:       11
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:       11
        inequality constraints with only lower bounds:       11
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

5694.536608985829